#### Follows official tutorial for classificatiion from tensorflow here [https://www.tensorflow.org/tutorials/images/classification?hl=zh-cn]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import pathlib

### first look at checking data

In [ ]:

data_dir = pathlib.Path('/Users/dhruvi/saltside/saltside-assets/v_data/train/')
image_count = len(list(data_dir.glob('*/*/*.jpg')))
# print(image_count)
cars = list(data_dir.glob('cars/*'))
PIL.Image.open(str(cars[0]))

In [ ]:
from keras.preprocessing.image import load_img
img = load_img(cars[0])
print(img.__array__)

In [ ]:
dir(img)

### Loading data using Keras utility

In [ ]:
batch_size = 32
img_height = 224
img_width = 224


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset="training",
    seed=123,
    image_size=(img_height,img_width),
    batch_size = batch_size
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset="validation",
    seed=123,
    image_size=(img_height,img_width),
    batch_size = batch_size
)

In [ ]:
class_names = train_ds.class_names
print(train_ds.class_names)


### Visualize the data

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

### Configure dataset for performance; so io doesnt become blocking

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)


### Standardize the data

In [ ]:
# you can apply a layer to a dataset with dataset.map or use that layer in model def
normalization_layer =layers.Rescaling(1. /255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x),y))
image_batch, label_batch = next(iter(normalized_ds))

plt.imshow(image_batch[0], cmap=plt.cm.binary)
plt.show()
print(np.min(image_batch[0]),np.max(image_batch[0]))

### Create a model

In [ ]:
num_classes = len(class_names)

model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height,img_width,3)),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes)
])

### Compile model

In [ ]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()

### Train the model

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


### Visualize the training results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epoch_range = range(epochs)

plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.plot(epoch_range, acc, label='Trainiing accuracy')
plt.plot(epoch_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and validation accuracy')

plt.subplot(1,2,2)
plt.plot(epoch_range, loss, label='Trainiing Loss')
plt.plot(epoch_range, val_loss, label = 'Validation Loss')
plt.legend(loc='lower right')
plt.title('Training and validation Loss')

plt.show()


##### Note: the model has achieved only 60% accuracy on the validation set; and the training and validation accuracy are off by large margin

### Getting results 

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal",
                     input_shape=(img_height,img_width,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [ ]:
num_classes = len(class_names)
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(img_height,img_width,3)),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes)
])

model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

### Predict on new data

In [ ]:
img = tf.keras.utils.load_img(
    '/Users/dhruvi/saltside/saltside-assets/v_data/onlineImages/8.jpeg',
    target_size=(img_height,img_width)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

plt.imshow(img)
plt.show()
print(f'Class: {class_names[np.argmax(score)]} Score: {100*np.max(score)}')

In [ ]:
import pprint
pprint.pprint(model.to_json())